In [2]:
import sqlite3
import pandas as pd
from notebook_tools import side_by_side

# import plotly.express as px
# import seaborn as sns

connection = sqlite3.connect("db/vivino.db")
cursor = connection.cursor()

In [3]:
from notebook_tools import get_sql_table_info

get_sql_table_info(cursor, save_to_file=True, print_out=True)



Table: countries
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	code                    VARCHAR      1       None      1    
	name                    VARCHAR      0       None      0    
	regions_count           INTEGER      0       None      0    
	users_count             INTEGER      0       None      0    
	wines_count             INTEGER      0       None      0    
	wineries_count          INTEGER      0       None      0    

Table: grapes
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	id                      INTEGER      1       None      1    
	name                    VARCHAR      0       None      0    

Table: wineries
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	id                      INTEGER      1       None      1    
	name                    VARCHAR      0       None      0    

Table: flavor_groups
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY

'\n\nTable: countries\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tcode                    VARCHAR      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\tregions_count           INTEGER      0       None      0    \n\tusers_count             INTEGER      0       None      0    \n\twines_count             INTEGER      0       None      0    \n\twineries_count          INTEGER      0       None      0    \n\nTable: grapes\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tid                      INTEGER      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\nTable: wineries\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tid                      INTEGER      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\nTable: flavor_groups\nColumns:\n\tNAME                    T

In [35]:
pd.read_sql_query(
"""
SELECT
    wines.name,
    acidity,
    fizziness,
    intensity,
    sweetness,
    tannin,
    regions.name,
    url
FROM
    wines
    JOIN regions ON regions.id = wines.region_id
WHERE
    wines.name = 'Cabernet Sauvignon'
""", connection)

,name,acidity,fizziness,intensity,sweetness,tannin,name,url
0,Cabernet Sauvignon,3.334017,None,4.709039,1.690490,3.371491,Napa Valley,https://www.grandcruwijnen.nl/2018-joseph-phel...
1,Cabernet Sauvignon,2.329016,None,3.928453,2.311101,2.895819,Veneto,https://www.seulementproduitsitaliens.be/vino/...
2,Cabernet Sauvignon,3.284369,None,4.748084,1.936713,3.146816,Napa Valley,https://www.wereldwijnonline.be/caymus-caberne...
3,Cabernet Sauvignon,3.869297,None,3.964489,1.540390,3.795511,Walla Walla Valley,https://www.seulementproduitsitaliens.be/spec_...
4,Cabernet Sauvignon,3.387184,None,4.585916,1.659149,3.429642,Napa Valley,https://www.grandcruwijnen.nl/2019-forman-napa...
5,Cabernet Sauvignon,3.884835,None,3.994474,1.644870,3.914068,Columbia Valley,https://www.grandcruwijnen.nl/2018-quilceda-cr...
6,Cabernet Sauvignon,3.304091,None,4.700800,1.758352,3.393993,Rutherford,https://www.seulementproduitsitaliens.be/spec_...
7,Cabernet Sauvignon,3.096885,None,4.536795,1.445688,3.653364,California,https://hermanwines.com/products/cabernet-sauv...


### Ratings overview: 10 most profitable wines?

In [2]:
q = """
SELECT
    wines.ratings_average AS 'avg rating',
    wines.ratings_count AS 'rating count'
FROM
    wines
"""

wine_df = pd.read_sql_query(q, connection)

q = """
SELECT
    vintages.ratings_average AS 'avg rating',
    vintages.ratings_count AS 'rating count'
FROM
    vintages
WHERE
    ratings_count AND ratings_average > 0
"""

vintages_df = pd.read_sql_query(q, connection)

side_by_side({
    "WINES Ratings": wine_df.describe(),
    # "WINES Correlation": wine_df.corr(),
    "VINTAGES Ratings": vintages_df.describe(),
    # "VINTAGES Correlation": vintages_df.corr(),
})

,avg rating,rating count
count,1020.000000,1020.000000
mean,4.443431,6387.334314
std,0.118239,14022.610539
min,4.100000,27.000000
25%,4.400000,474.000000
50%,4.400000,1697.500000
75%,4.500000,5644.750000
max,4.800000,157944.000000
,avg rating,rating count
count,1540.000000,1540.000000


In [3]:
q = """
SELECT
    wines.name AS Wine,
    --wines.id AS ID,
    vintages.name AS Vintage,
    CAST(AVG(vintages.price_euros) AS INT) AS 'Price €',
    wines.ratings_average AS Rating,
    SUM(wines.ratings_count) AS Count,
    CAST(wines.ratings_average * LOG(wines.ratings_count * 0.0001) * vintages.price_euros / 10 AS INT) AS Score,
    regions.name AS Region,
    countries.name AS Country,
    vtr.rank
FROM
    wines
    JOIN vintages ON vintages.wine_id = wines.id
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
    JOIN vintage_toplists_rankings vtr ON vtr.vintage_id = vintages.id
GROUP BY
    wines.id
ORDER BY
    Score DESC
LIMIT 10
"""

pd.read_sql_query(q, connection).head(60)

,Wine,Vintage,Price €,Rating,Count,Score,Region,Country,rank
0,Sassicaia,Tenuta San Guido Sassicaia 1988,2668,4.6,107646,1266,Bolgheri Sassicaia,Italie,2
1,Cristal Brut Champagne (Millésimé),Louis Roederer Cristal Brut Champagne (Millési...,1900,4.6,84912,548,Champagne,France,5
2,Pauillac (Premier Grand Cru Classé),Château Lafite Rothschild Pauillac (Premier Gr...,1900,4.6,78566,519,Pauillac,France,2
3,Toscana,Masseto Toscana 2006,4633,4.7,81420,488,Toscana,Italie,1
4,Grand Vin Pauillac (Premier Grand Cru Classé),Château Latour Grand Vin Pauillac (Premier Gra...,1505,4.6,77042,454,Pauillac,France,2
5,Pauillac (Premier Grand Cru Classé),Château Mouton Rothschild Pauillac (Premier Gr...,1038,4.6,73615,414,Pauillac,France,3
6,Château Margaux (Premier Grand Cru Classé),Château Margaux Château Margaux (Premier Grand...,1077,4.6,45350,325,Margaux,France,1
7,Tignanello,Antinori Tignanello 2016,620,4.5,285026,218,Toscana,Italie,10
8,Vintage,Krug Vintage 2002,1691,4.6,50754,189,Champagne,France,7
9,Pauillac (Grand Cru Classé),Château Pontet-Canet Pauillac (Grand Cru Class...,438,4.4,141702,177,Pauillac,France,5


### Marketing budget: which country?

In [6]:
q = """
SELECT
    countries.name,
    countries.users_count,
    countries.users_count / wineries_count AS users_per_winery
FROM
    countries
ORDER BY countries.users_count DESC
LIMIT 10
"""

df = pd.read_sql_query(q, connection)
df

q = """
SELECT
    countries.name,
    countries.users_count,
    countries.users_count / wineries_count AS users_per_winery
FROM
    countries
ORDER BY users_per_winery DESC
LIMIT 10
"""

df2 = pd.read_sql_query(q, connection)
df2

side_by_side({
    "Ordered by user count": df,
    "Ordered by users/winery": df2
})

,name,users_count,users_per_winery
0,États-Unis,12273684,436
1,France,5973301,88
2,Italie,4270717,100
3,Allemagne,2549989,186
4,Espagne,2264396,125
5,Suisse,1601799,416
6,Portugal,1123535,192
7,Australie,1022965,73
8,Argentine,629532,117
9,Chili,326757,56


### 3 Relevant Wineries

In [86]:
q = """
WITH temp2 AS (
    WITH temp AS (
        SELECT
            vintages.name AS Vintage,
            wines.name AS Wine,
            vtr.rank AS Rank,
            vintages.ratings_average AS Ratings,
            vintages.ratings_count AS Count,
            vintages.year as Year
        FROM
            vintages
            JOIN wines ON vintages.wine_id = wines.id
            LEFT JOIN vintage_toplists_rankings AS vtr ON vtr.vintage_id = vintages.id
        WHERE
            vintages.year != 'N.V.'
            AND vintages.year > 2010
            AND Ratings != 0
            --AND Rank != 'None'
    )
    SELECT
        SUBSTR(Vintage, 1, LENGTH(Vintage) - LENGTH(Wine) - 6) AS Winery,
        MAX(Year) AS LatestYear,
        --Wine,
        --Vintage,
        --MIN(Rank) AS 'Best Rank',
        MAX(Ratings) AS Ratings,
        SUM(Count) AS Count
    FROM temp
    GROUP BY
        Winery
    ORDER BY
        Count DESC
)
SELECT
    Winery,
    LatestYear AS 'Latest Year',
    Ratings AS 'Best Rating',
    Count AS Popularity,
    Ratings * LOG(Count * 0.000001 + 1.5) AS Score
FROM
    temp2
ORDER BY
    Score DESC
LIMIT 3
"""

df = pd.read_sql_query(q, connection)
side_by_side({
    "": df
})

,Winery,Latest Year,Best Rating,Popularity,Score
0,Antinori,2021,4.700000,75989,0.928500
1,Tenuta San Guido,2020,4.700000,48554,0.892654
2,Casanova di Neri,2018,4.800000,17436,0.869330


### Coffee, Toast, Cream, Green Apple and Citrus

In [51]:
q = """
WITH concat_keywords AS (
    SELECT
        wines.name AS Wine,
        wines.ratings_average AS Rating,
        keywords_wine.count AS count,
        keywords.name AS keyword,
        GROUP_CONCAT(keywords.name) AS AllKeywords
    FROM
        keywords_wine
        JOIN wines ON keywords_wine.wine_id = wines.id
        JOIN keywords ON keywords_wine.keyword_id = keywords.id
    WHERE
        keyword IN ("coffee", "toast", "green apple", "cream", "citrus")
        AND count > 9
    GROUP BY
        wines.id
)
SELECT
    Wine,
    AllKeywords,
    Rating
FROM
    concat_keywords
WHERE
    AllKeywords LIKE '%coffee%'
    AND AllKeywords LIKE '%toast%'
    AND AllKeywords LIKE '%green apple%'
    AND AllKeywords LIKE '%cream%'
    AND AllKeywords LIKE '%citrus%'
ORDER BY
    Rating DESC
"""

df = pd.read_sql_query(q, connection)
df.head(60)

,Wine,AllKeywords,Rating
0,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,"citrus,toast,coffee,coffee,cream,toast,green a...",4.7
1,Sauternes,"green apple,coffee,toast,coffee,citrus,cream,t...",4.7
2,P2 Plénitude Brut Champagne,"toast,coffee,coffee,cream,toast,green apple,ci...",4.7
3,Cristal Brut Champagne (Millésimé),"toast,coffee,green apple,citrus,coffee,cream,t...",4.6
4,Vintage,"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
5,Brut Champagne,"toast,coffee,cream,toast,coffee,green apple,ci...",4.6
6,R.D Extra Brut Champagne (Récemment Dégorgé),"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
7,Blanc des Millénaires,"toast,coffee,coffee,cream,toast,citrus,green a...",4.6
8,Sir Winston Churchill Brut Champagne,"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
9,Grande Cuvée,"toast,coffee,coffee,cream,toast,citrus,green a...",4.6


### TOP 5 Wines for the TOP 3 Grapes

In [71]:
q = """
WITH ranked_wines AS (
    SELECT
        wines.name AS Wine,
        grapes.name AS Grape,
        wines.ratings_average AS 'Rating',
        countries.name AS Country,
        ROW_NUMBER() OVER(PARTITION BY grapes.name ORDER BY wines.ratings_average DESC) AS wine_rank
    FROM
        wines
        JOIN regions ON wines.region_id = regions.id
        JOIN countries ON regions.country_code = countries.code
        JOIN most_used_grapes_per_country AS mugpc ON countries.code = mugpc.country_code
        JOIN grapes ON grapes.id = mugpc.grape_id
    WHERE
        grapes.name IN (
            SELECT
                distinct(name)
            FROM
                most_used_grapes_per_country AS mugpc
                JOIN grapes ON grapes.id = mugpc.grape_id
            ORDER BY
                wines_count DESC
            LIMIT 3
        )
    ORDER BY
        wines.ratings_average DESC
)
SELECT
    Wine,
    Grape,
    Country,
    Rating
FROM
    ranked_wines
WHERE
    wine_rank <= 5
ORDER BY
    Grape, Rating DESC;
"""

wines_df = pd.read_sql_query(q, connection)
wines_df.head(50)

,Wine,Grape,Country,Rating
0,Cabernet Sauvignon,Cabernet Sauvignon,États-Unis,4.8
1,Mágico,Cabernet Sauvignon,Espagne,4.8
2,IX Estate Red,Cabernet Sauvignon,États-Unis,4.7
3,Special Selection Cabernet Sauvignon,Cabernet Sauvignon,États-Unis,4.7
4,Unico Reserva Especial Edición,Cabernet Sauvignon,Espagne,4.7
5,Amarone della Valpolicella Classico Riserva,Chardonnay,Italie,4.8
6,Cabernet Sauvignon,Chardonnay,États-Unis,4.8
7,Fratini Bolgheri Superiore,Chardonnay,Italie,4.8
8,Cristal Rosé Vinothèque,Chardonnay,France,4.8
9,Batard-Montrachet Grand Cru,Chardonnay,France,4.8


### AVG Wine rating for each country and each vintage

In [68]:
q = """
SELECT
    avg(wines.ratings_average) AS AverageRating,
    sum(wines.ratings_count) AS RatingCount,
    countries.name AS Country
FROM
    wines
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
WHERE
    wines.ratings_average AND wines.ratings_count > 0
GROUP BY
    Country
ORDER BY
    AverageRating DESC
"""

wine_df = pd.read_sql_query(q, connection)

q = """
SELECT
    avg(vintages.ratings_average) AS AverageRating,
    sum(vintages.ratings_count) AS RatingCount,
    countries.name AS Country
FROM
    vintages
    JOIN wines ON wines.id = vintages.wine_id
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
WHERE
    vintages.ratings_average AND vintages.ratings_count > 0
GROUP BY
    Country
ORDER BY
    AverageRating DESC
"""

vintages_df = pd.read_sql_query(q, connection)

side_by_side({
    "Wines": wine_df,
    "Vintages": vintages_df
})


,AverageRating,RatingCount,Country
0,4.500000,894,Israël
1,4.500000,8257,Allemagne
2,4.490541,834263,États-Unis
3,4.480000,14114,Moldavie
4,4.472727,30564,Hongrie
5,4.459091,59995,Afrique du Sud
6,4.458333,52138,Australie
7,4.447130,2124809,France
8,4.443617,746710,Espagne
9,4.435714,62454,Portugal


### TOP 5 Cabernet Sauvignon pour Monsieur Vielle-Pie

In [63]:
q = """
SELECT
    wines.name AS Wine,
    grapes.name AS Grape,
    wines.ratings_average AS Rating,
    wines.ratings_count AS 'Ratings Count',
    wines.ratings_average * LOG(wines.ratings_count * 0.000001 + 2.2) AS Popularity
FROM
    wines
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
    JOIN most_used_grapes_per_country AS mugpc ON mugpc.country_code = countries.code
    JOIN grapes ON grapes.id = mugpc.grape_id
WHERE Grape = 'Cabernet Sauvignon'
ORDER BY
    Popularity DESC
LIMIT 5
"""

df = pd.read_sql_query(q, connection)
df

,Wine,Grape,Rating,Ratings Count,Popularity
0,Cabernet Sauvignon,Cabernet Sauvignon,4.6,157944,1.713654
1,Unico,Cabernet Sauvignon,4.7,45140,1.650844
2,Special Selection Cabernet Sauvignon,Cabernet Sauvignon,4.7,41236,1.647292
3,Cabernet Sauvignon,Cabernet Sauvignon,4.8,2941,1.646414
4,Opus One,Cabernet Sauvignon,4.6,77053,1.643916
